In [1]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import re
import unicodedata
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

BASE_DIR = Path().resolve().parent.parent

In [2]:
def fill_district(truocsapnhap):
    parts = truocsapnhap.split(', ')
    current_district = None
    filled = []

    # Loop từ phải qua trái
    for part in reversed(parts):
        # Tìm district trong ngoặc
        m = re.search(r'\((.*?)\)', part)
        if m:
            current_district = m.group(1)
        # Nếu chưa có district thì bỏ qua
        if current_district:
            # Bỏ ngoặc cũ nếu có
            part = re.sub(r'\(.*?\)', '', part).strip()
            part = f"{part} ({current_district})"
        filled.append(part)

    # Đảo lại đúng thứ tự ban đầu
    filled = list(reversed(filled))
    return ', '.join(filled)


def extract_district(truocsapnhap):
    unit = truocsapnhap.split(' (')
    district = np.nan
    if len(unit) > 1:
        district = unit[1][:-1]
        if ' - Tỉnh ' in district:
            district = district.split(' - Tỉnh ')[0]
    return district


def extract_province(truocsapnhap):
    unit = truocsapnhap.split(' (')
    province = np.nan
    if len(unit) > 1:
        district = unit[1][:-1]
        if ' - Tỉnh ' in district:
            province = district.split(' - ')[1]
    return province


def extract_ward(truocsapnhap):
    unit = truocsapnhap.split(' (')
    ward = unit[0]
    return ward


def cap_first(text):
    '''
    Capitalize the first character of a string
    :param text: str
    :return: str or np.nan
    '''
    if isinstance(text, str):
        split_text = text.split()
        split_text = [i[0].capitalize() + i[1:] if split_text.index(i) == 0 else i for i in split_text]
        return ' '.join(split_text)
    return np.nan


def lower_safely(text):
    '''
    Lower string ignore NaN
    :param text:
    :return:
    '''
    if isinstance(text, str):
        return text.lower()
    return np.nan


def unidecode_pro(text):
    '''
    Xóa dấu tiếng Việt và các ký tự ngoài số và chữ
    :param text: str
    :return: str or np.nan
    '''
    if isinstance(text, str):
        text = unidecode(text)
        text = re.sub(r'[^a-zA-Z0-9\,]+', ' ', text)  # Chỉ giữ a-z, A-Z, 0-9 và dấu phẩy, thay phần còn lại bằng khoảng trắng
        text = re.sub(r'\s+', ' ', text) # Strip double space
        return text.strip().lower()
    return np.nan


def unicode_normalize(text):
    '''
    Chuyển Unicode tổ hợp sang Unicode dựng sẵn, fix lỗi gõ dấu bằng ký tự đặc biệt
    :param text: str
    :return: str or np.nan
    '''
    if isinstance(text, str):
        text = unicodedata.normalize('NFC', text)
        text = text.replace("’", "'").replace("‘", "'").replace("“", '"').replace("”", '"').strip() # Normalize quotes
        return text
    return text

# Chuẩn bị danh mục 63 tỉnh thành

In [3]:
# Ward level is not includes 5 islands
df_danhmuc_ward = pd.read_csv(BASE_DIR /'data/danhmuchanhchinh.gso.gov.vn_2025-07-18.csv')
rename_cols = {
    'Mã': 'wardCode',
    'Mã QH': 'districtCode',
    'Mã TP': 'provinceCode',
    'Cấp': 'wardType',
    'Tên': 'ward',
    'Quận Huyện': 'district',
    'Tỉnh / Thành Phố': 'province'
}
df_danhmuc_ward = df_danhmuc_ward.rename(columns=rename_cols)[rename_cols.values()]

In [4]:
# District level includes 5 islands
df_danhmuc_district = pd.read_csv(BASE_DIR / 'data/danhmuchanhchinh.gso.gov.vn_district_2025-07-18.csv')
rename_cols = {
    'Mã': 'districtCode',
    'Mã TP': 'provinceCode',
    'Cấp': 'districtType',
    'Tên': 'district',
    'Tỉnh / Thành Phố': 'province'
}

df_danhmuc_district = df_danhmuc_district.rename(columns=rename_cols)[rename_cols.values()]

In [5]:
# 63 provinces units
common = list(set(df_danhmuc_ward.columns.tolist()) & set(df_danhmuc_district.columns.tolist()))
df_danhmuc = pd.merge(df_danhmuc_district, df_danhmuc_ward, on=common, how='left')

In [6]:
print('df_danhmuc_ward.shape:', df_danhmuc_ward.shape)
print('df_danhmuc_district.shape:', df_danhmuc_district.shape)
print('df_danhmuc.shape:', df_danhmuc.shape)

df_danhmuc_ward.shape: (10035, 7)
df_danhmuc_district.shape: (696, 5)
df_danhmuc.shape: (10040, 8)


In [7]:
# Check NaN wards, It would be an island district
df_danhmuc[df_danhmuc['ward'].isna()]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward
3582,318,31,Huyện,Huyện Bạch Long Vĩ,Thành phố Hải Phòng,NaN,NaN,NaN
5794,471,45,Huyện,Huyện Cồn Cỏ,Tỉnh Quảng Trị,NaN,NaN,NaN
5975,498,48,Huyện,Huyện Hoàng Sa,Thành phố Đà Nẵng,NaN,NaN,NaN
6379,536,51,Huyện,Huyện Lý Sơn,Tỉnh Quảng Ngãi,NaN,NaN,NaN
8196,755,77,Huyện,Huyện Côn Đảo,Tỉnh Bà Rịa - Vũng Tàu,NaN,NaN,NaN


# Sáp nhập province

In [8]:
df_sapnhap_province = pd.read_csv(BASE_DIR / 'data/sapnhap.bando.com.vn_province.csv')

# Enrich data
df_sapnhap_province['isMergedProvince'] = np.where(df_sapnhap_province['truocsapnhap']=='không sáp nhập', False, True)

# Unpivot
df_sapnhap_province['truocsapnhap'] = df_sapnhap_province['truocsapnhap'].str.replace(' và ', ', ')
df_sapnhap_province['truocsapnhap'] = df_sapnhap_province['truocsapnhap'].str.split(',\s*')
df_sapnhap_province = df_sapnhap_province.explode('truocsapnhap').reset_index(drop=True)

# Fill un-merged province name for truocsapnhap
df_sapnhap_province['truocsapnhap'] = np.where(df_sapnhap_province['isMergedProvince']==True, df_sapnhap_province['truocsapnhap'], df_sapnhap_province['tentinh'])

# Capitalize the first character
df_sapnhap_province['truocsapnhap'] = df_sapnhap_province['truocsapnhap'].apply(cap_first)
df_sapnhap_province['tentinh'] = df_sapnhap_province['tentinh'].apply(cap_first)

In [9]:
df_sapnhap_province

,id,mahc,tentinh,dientichkm2,dansonguoi,trungtamhc,kinhdo,vido,truocsapnhap,con,isMergedProvince
0,1,1,Thủ đô Hà Nội,"3.359,80",8.718.000,giữ nguyên,105.698,21.00010,Thủ đô Hà Nội,126 ĐVHC (51 phường và 75 xã),False
1,2,7,Tỉnh Cao Bằng,"6.700,40",555.809,giữ nguyên,106.083,22.74560,Tỉnh Cao Bằng,"56 ĐVHC (03 phường, 53 xã)",False
2,3,8,Tỉnh Tuyên Quang,"13.795,50",1.865.270,Tuyên Quang (cũ),105.099,22.48970,Tỉnh Hà Giang,"124 ĐVHC (07 phường, 117 xã)\n",True
3,3,8,Tỉnh Tuyên Quang,"13.795,50",1.865.270,Tuyên Quang (cũ),105.099,22.48970,Tỉnh Tuyên Quang,"124 ĐVHC (07 phường, 117 xã)\n",True
4,4,9,Tỉnh Lào Cai,"13.256,92",1.778.785,Yên Bái (cũ),104.347,22.06050,Tỉnh Yên Bái,"99 ĐVHC (10 phường, 89 xã)",True
...,...,...,...,...,...,...,...,...,...,...,...
58,33,33,Thành phố Cần Thơ,"6.360,83",4.199.824,Cần Thơ (cũ),105.757,9.74372,Thành phố Cần Thơ,103 ĐVHC (31 phường và 72 xã),True
59,33,33,Thành phố Cần Thơ,"6.360,83",4.199.824,Cần Thơ (cũ),105.757,9.74372,Tỉnh Sóc Trăng,103 ĐVHC (31 phường và 72 xã),True
60,33,33,Thành phố Cần Thơ,"6.360,83",4.199.824,Cần Thơ (cũ),105.757,9.74372,Tỉnh Hậu Giang,103 ĐVHC (31 phường và 72 xã),True
61,34,34,Tỉnh Cà Mau,"7.942,39",2.606.672,Cà mau (cũ),105.182,9.13620,Tỉnh Bạc Liêu,"64 ĐVHC (09 phường, 55 xã)",True


In [10]:
# Check missmatch province
df_sapnhap_province[~df_sapnhap_province['truocsapnhap'].isin(df_danhmuc['province'])]

,id,mahc,tentinh,dientichkm2,dansonguoi,trungtamhc,kinhdo,vido,truocsapnhap,con,isMergedProvince
0,1,1,Thủ đô Hà Nội,"3.359,80",8.718.000,giữ nguyên,105.698,21.0001,Thủ đô Hà Nội,126 ĐVHC (51 phường và 75 xã),False
14,11,12,Tỉnh Phú Thọ,"9.361,38",4.022.638,Phú Thọ (cũ),105.333,20.7065,Tỉnh Hòa Bình,"148 ĐVHC (15 phường, 133 xã)",True
48,29,29,Thành phố Hồ Chí Minh,"6.772,59",14.002.598,Tp. HCM (cũ),106.638,10.9926,TPHCM,"168 ĐVHC (01 đặc khu, 113 phường, 54 xã)",True


In [11]:
# Correct province name follow df_danhmuc
df_sapnhap_province.loc[df_sapnhap_province['truocsapnhap']=='Thủ đô Hà Nội', 'truocsapnhap'] = 'Thành phố Hà Nội'
df_sapnhap_province.loc[df_sapnhap_province['truocsapnhap']=='TPHCM', 'truocsapnhap'] = 'Thành phố Hồ Chí Minh'
df_sapnhap_province.loc[df_sapnhap_province['truocsapnhap']=='Tỉnh Hòa Bình', 'truocsapnhap'] = 'Tỉnh Hoà Bình'

In [12]:
# Merge to df_danhmuc
rename_cols = {
    'truocsapnhap': 'province',
    'mahc': 'newProvinceCode',
    'tentinh': 'newProvince',
    'vido': 'newProvinceLat',
    'kinhdo': 'newProvinceLon',
    'isMergedProvince': 'isMergedProvince',
}
df_sapnhap_province = df_sapnhap_province.rename(columns=rename_cols)[rename_cols.values()]

df_danhmuc = pd.merge(df_danhmuc, df_sapnhap_province, on='province', how='left')

In [13]:
df_danhmuc

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,newProvinceLat,newProvinceLon,isMergedProvince
0,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,1.0,Phường,Phường Phúc Xá,1,Thủ đô Hà Nội,21.0001,105.698,False
1,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,4.0,Phường,Phường Trúc Bạch,1,Thủ đô Hà Nội,21.0001,105.698,False
2,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,6.0,Phường,Phường Vĩnh Phúc,1,Thủ đô Hà Nội,21.0001,105.698,False
3,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,7.0,Phường,Phường Cống Vị,1,Thủ đô Hà Nội,21.0001,105.698,False
4,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,8.0,Phường,Phường Liễu Giai,1,Thủ đô Hà Nội,21.0001,105.698,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10035,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32239.0,Xã,Xã Viên An Đông,34,Tỉnh Cà Mau,9.1362,105.182,True
10036,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32242.0,Xã,Xã Viên An,34,Tỉnh Cà Mau,9.1362,105.182,True
10037,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32244.0,Thị trấn,Thị trấn Rạch Gốc,34,Tỉnh Cà Mau,9.1362,105.182,True
10038,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32245.0,Xã,Xã Tân Ân,34,Tỉnh Cà Mau,9.1362,105.182,True


In [14]:
if df_danhmuc['newProvince'].isna().sum():
    raise Exception('There are mismatched provinces')

In [15]:
df_danhmuc

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,newProvinceLat,newProvinceLon,isMergedProvince
0,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,1.0,Phường,Phường Phúc Xá,1,Thủ đô Hà Nội,21.0001,105.698,False
1,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,4.0,Phường,Phường Trúc Bạch,1,Thủ đô Hà Nội,21.0001,105.698,False
2,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,6.0,Phường,Phường Vĩnh Phúc,1,Thủ đô Hà Nội,21.0001,105.698,False
3,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,7.0,Phường,Phường Cống Vị,1,Thủ đô Hà Nội,21.0001,105.698,False
4,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,8.0,Phường,Phường Liễu Giai,1,Thủ đô Hà Nội,21.0001,105.698,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10035,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32239.0,Xã,Xã Viên An Đông,34,Tỉnh Cà Mau,9.1362,105.182,True
10036,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32242.0,Xã,Xã Viên An,34,Tỉnh Cà Mau,9.1362,105.182,True
10037,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32244.0,Thị trấn,Thị trấn Rạch Gốc,34,Tỉnh Cà Mau,9.1362,105.182,True
10038,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32245.0,Xã,Xã Tân Ân,34,Tỉnh Cà Mau,9.1362,105.182,True


# Sáp nhập ward

## Unpivot

In [16]:
df_sapnhap_ward = pd.read_csv(BASE_DIR / 'data/sapnhap.bando.com.vn_ward.csv')

# Remove và replace text
# df_sapnhap_ward[df_sapnhap_ward['truocsapnhap'].str.contains('\(')]['truocsapnhap']
removed_texts = [
    ', Trường bắn TB1',
    ', Khu vực bãi bồi ven biển \(do huyện Kim Sơn quản lý\)',
    '\(phần còn lại.*?\)',
    '\(bao gồm.*?\)',
    '\(một phần.*?\)',
    '\(do huyện.*?\)',
]

replaced_texts = [
    (' và ', ', '),
    ('(thuộc huyện ', '(huyện '),

    # Chỉ có 2 huyện này có thông tin tỉnh
    # Nên giữ tỉnh thành để merge không bị duplicate Thị trấn Châu Thành, Huyện Châu Thành
    ('(huyện Châu Thành, tỉnh Bến Tre)', '(Huyện Châu Thành - Tỉnh Bến Tre)'),
    ('(huyện Châu Thành, tỉnh Trà Vinh)', '(Huyện Châu Thành - Tỉnh Trà Vinh)')
]

for text in removed_texts:
    df_sapnhap_ward['truocsapnhap'] = df_sapnhap_ward['truocsapnhap'].str.replace(text, '', regex=True).str.strip()

for old, new in replaced_texts:
    df_sapnhap_ward['truocsapnhap'] = df_sapnhap_ward['truocsapnhap'].str.replace(old, new, regex=False).str.strip()

# Capitalize the first character for province
df_sapnhap_ward['tentinh'] = df_sapnhap_ward['tentinh'].apply(cap_first)

# Correct name for province (Must do after capitalizing)
df_sapnhap_ward['tentinh'].replace('Tỉnh Ninh BÌnh', 'Tỉnh Ninh Bình', inplace=True)

In [17]:
# Add isMergedWard
df_sapnhap_ward['isMergedWard'] = np.where(df_sapnhap_ward['truocsapnhap']=='Không sáp nhập', False, True)

# Unpivot
df_sapnhap_ward['truocsapnhap'] = df_sapnhap_ward['truocsapnhap'].apply(fill_district)
df_sapnhap_ward['truocsapnhap'] = df_sapnhap_ward['truocsapnhap'].str.split(',\s*')
df_sapnhap_ward = df_sapnhap_ward.explode('truocsapnhap').reset_index(drop=True)

In [18]:
# Extract ward, district, province from truocsapnhap
df_sapnhap_ward['province'] = df_sapnhap_ward['truocsapnhap'].apply(extract_province)
df_sapnhap_ward['district'] = df_sapnhap_ward['truocsapnhap'].apply(extract_district)
df_sapnhap_ward['ward'] = df_sapnhap_ward['truocsapnhap'].apply(extract_ward)

# Fill un-merged ward name for truocsapnhap
df_sapnhap_ward['ward'] = np.where(df_sapnhap_ward['isMergedWard']==False, df_sapnhap_ward['loai'].str.capitalize() + ' ' + df_sapnhap_ward['tenhc'], df_sapnhap_ward['ward'])

# Capitalize the first character for ward and district
df_sapnhap_ward['ward'] = df_sapnhap_ward['ward'].apply(cap_first)
df_sapnhap_ward['district'] = df_sapnhap_ward['district'].apply(cap_first)

In [19]:
# Bring district value in ward column to district column
# df_sapnhap_ward[df_sapnhap_ward['ward'].str.contains('^Huyện')]
df_sapnhap_ward['district'] = np.where(df_sapnhap_ward['ward'].str.contains('^Huyện'), df_sapnhap_ward['ward'], df_sapnhap_ward['district'])
df_sapnhap_ward['ward'] = np.where(df_sapnhap_ward['ward'].str.contains('^Huyện'), np.nan, df_sapnhap_ward['ward'])

In [20]:
# Rename for next steps
rename_cols = {
    'province': 'province',
    'district': 'district',
    'ward': 'ward',
    'isMergedWard': 'isMergedWard',

    'tenhc': 'newWard',
    'tentinh': 'newProvince',
    'ma': 'newWardCode',
    'loai': 'newWardType',
    'kinhdo': 'newWardLon',
    'vido': 'newWardLat',
    'dientichkm2': 'newWardAreaKm2',
    'truocsapnhap': 'truocsapnhap',

}
df_sapnhap_ward = df_sapnhap_ward.rename(columns=rename_cols)[rename_cols.values()]

In [21]:
# Check 2 provinces have "Huyện Châu Thành"
df_sapnhap_ward[~df_sapnhap_ward['province'].isna()]

,province,district,ward,isMergedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap
9222,Tỉnh Trà Vinh,Huyện Châu Thành,Thị trấn Châu Thành,True,Châu Thành,Tỉnh Vĩnh Long,2880,xã,106.337,9.83937,47.71,Thị trấn Châu Thành (Huyện Châu Thành - Tỉn...
9374,Tỉnh Bến Tre,Huyện Châu Thành,Thị trấn Châu Thành,True,Phú Túc,Tỉnh Vĩnh Long,2934,xã,106.318,10.30100,63.14,Thị trấn Châu Thành (Huyện Châu Thành - Tỉn...


In [22]:
if df_sapnhap_ward[~df_sapnhap_ward['newProvince'].isin(df_danhmuc['newProvince'])].shape[0]:
    raise Exception('There are mismatched provinces')
# df_sapnhap_ward[~df_sapnhap_ward['newProvince'].isin(df_danhmuc['newProvince'])]['newProvince'].drop_duplicates()

In [23]:
df_sapnhap_ward

,province,district,ward,isMergedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap
0,NaN,NaN,Xã Đông La,True,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Xã Đông La
1,NaN,NaN,Phường Dương Nội,True,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Phường Dương Nội
2,NaN,NaN,Xã An Khánh,True,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Xã An Khánh
3,NaN,NaN,Xã La Phù,True,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Xã La Phù
4,NaN,NaN,Xã Song Phương,True,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Xã Song Phương
...,...,...,...,...,...,...,...,...,...,...,...,...
10562,NaN,NaN,Xã Vĩnh Phú Tây,True,Vĩnh Phước,Tỉnh Cà Mau,3353,xã,105.410,9.39891,75.50,Xã Vĩnh Phú Tây
10563,NaN,NaN,Xã Hưng Phú,True,Vĩnh Thanh,Tỉnh Cà Mau,3354,xã,105.519,9.35997,37.37,Xã Hưng Phú
10564,NaN,NaN,Xã Vĩnh Thanh,True,Vĩnh Thanh,Tỉnh Cà Mau,3354,xã,105.519,9.35997,37.37,Xã Vĩnh Thanh
10565,NaN,Thành phố Bạc Liêu,Phường 5,True,Vĩnh Trạch,Tỉnh Cà Mau,3355,phường,105.774,9.29729,10.22,Phường 5 (thành phố Bạc Liêu)


## Kiểm tra chính tả 2 bên

Vòng lặp kiểm tra như sau:
- Bước 1: Kiểm tra tìm unit không khớp của bảng df_sapnhap_ward
- Bước 2: Sửa chính tả ở bảng df_sapnhap_ward hoặc df_danhmuc.
- Quay lại Bước 1

In [24]:
def correct_basic_spelling(text):
    basic_spelling = [
        ('Hoà', 'Hòa'),
        ('Hoè', 'Hòe'),
        ('Hoá', 'Hóa'),
        ('Hoả', 'Hỏa'),
        ('Khoá', 'Khóa'),
        ('Thuỵ' ,'Thụy'),
        ('Thuỷ', 'Thủy'),
        ('\' ', "'")
    ]

    if isinstance(text, str):
        for wrong, right in basic_spelling:
            pattern = r'\b' + wrong + r'\b'
            text = re.sub(pattern, right, text)
    return text

In [25]:
# Correct spelling
for col in ['province', 'district', 'ward']:
    # Convert Unicode
    df_danhmuc[col] = df_danhmuc[col].apply(unicode_normalize)
    df_sapnhap_ward[col] = df_sapnhap_ward[col].apply(unicode_normalize)
    # Correct basic spelling
    df_danhmuc[col] = df_danhmuc[col].apply(correct_basic_spelling)
    df_sapnhap_ward[col] = df_sapnhap_ward[col].apply(correct_basic_spelling)


spelling_danhmuc = [
    # newProvince, ward, correct ward
    ('Tỉnh Hưng Yên', 'Xã An Quí', 'Xã An Quý'),
    ('Tỉnh Hưng Yên', 'Xã Kim Chung', 'Xã Kim Trung'),
    ('Tỉnh Hưng Yên', 'Xã Vũ Quí', 'Xã Vũ Quý'),
    ('Tỉnh Tuyên Quang', 'Xã Quí Quân', 'Xã Quý Quân'),
    ('Tỉnh Tuyên Quang', 'Xã Sán Xả Hồ', 'Xã Sán Sả Hồ'),
    ('Tỉnh Tuyên Quang', 'Xã Xà Phìn', 'Xã Sà Phìn'),
    ('Tỉnh Tuyên Quang', 'Xã Sủng Tráng', 'Xã Sủng Cháng'),
    ('Tỉnh Lào Cai', 'Xã Hát Lìu', 'Xã Hát Lừu'),
    ('Tỉnh Lào Cai', 'Thị trấn NT Liên Sơn', 'Thị trấn Nông trường Liên Sơn'),
    ('Tỉnh Lào Cai', 'Xã Nậm Rạng', 'Xã Nậm Dạng'),
    ('Tỉnh Thái Nguyên', 'Xã Sĩ Bình', 'Xã Sỹ Bình'),
    ('Tỉnh Lạng Sơn', 'Thị trấn NT Thái Bình', 'Thị trấn Nông trường Thái Bình'),
    ('Tỉnh Phú Thọ', 'Xã Săm Khóe', 'Xã Xăm Khòe'),
    ('Tỉnh Điện Biên', 'Xã Xá Tổng', 'Xã Sá Tổng'),
    ('Tỉnh Điện Biên', 'Xã Huổi Lếnh', 'Xã Huổi Lếch'),
    ('Tỉnh Điện Biên', 'Xã Sáng Nhè', 'Xã Xá Nhè'),
    ('Tỉnh Điện Biên', 'Xã Xín Chải', 'Xã Sín Chải'),
    ('Tỉnh Lai Châu', 'Xã Tông Qua Lìn', 'Xã Tung Qua Lìn'),
    ('Tỉnh Lai Châu', 'Xã Lả Nhì Thàng', 'Xã Lản Nhì Thàng'),
    ('Tỉnh Lai Châu', 'Xã Hua Bun', 'Xã Hua Bum'),
    ('Tỉnh Lai Châu', 'Xã Ma Ly Pho', 'Xã Ma Li Pho'),
    ('Tỉnh Sơn La', 'Xã Pú Pẩu', 'Xã Pú Bẩu'),
    ('Tỉnh Sơn La', 'Xã Noong Lay', 'Xã Nong Lay'),
    ('Tỉnh Nghệ An', 'Phường Thu Thuỷ', 'Phường Thu Thủy'),
    ('Tỉnh Nghệ An', 'Xã Hưng Yên', 'Xã Hưng Yên Nam'),
    ('Tỉnh Quảng Trị', 'Xã Đa Krông', 'Xã Đakrông'),
    ('Tỉnh Quảng Trị', 'Thị trấn NT Lệ Ninh', 'Thị trấn Nông trường Lệ Ninh'),
    ('Tỉnh Quảng Trị', 'Thị trấn NT Việt Trung', 'Thị trấn Nông trường Việt Trung'),
    ('Thành phố Đà Nẵng', 'Xã Bha Lê', 'Xã Bha Lêê'),
    ('Thành phố Đà Nẵng', 'Xã Tà Bhinh', 'Xã Tà Bhing'),
    ('Thành phố Đà Nẵng', 'Xã Mà Cooi', 'Xã Mà Cooih'),
    ('Thành phố Đà Nẵng', 'Thị trấn P Rao', 'Thị Trấn Prao'),
    ('Thành phố Đà Nẵng', 'Xã Laêê', 'Xã La Êê'),
    ('Thành phố Đà Nẵng', 'Xã Zuôich', 'Xã Zuôih'),
    ('Tỉnh Quảng Ngãi', 'Xã Bờ Y', 'Xã Pờ Y'),
    ('Tỉnh Quảng Ngãi', 'Xã Măng Buk', 'Xã Măng Bút'),
    ('Tỉnh Quảng Ngãi', 'Xã Ngọc Yêu', 'Xã Ngọk Yêu'),
    ('Tỉnh Quảng Ngãi', 'Xã Ngọc Lây', 'Xã Ngọk Lây'),
    ('Tỉnh Gia Lai', 'Xã Đăk Roong', 'Xã Đăk Rong'),
    ('Tỉnh Gia Lai', 'Xã H Bông', 'Xã HBông'),
    ('Tỉnh Gia Lai', 'Xã Ia Ma Rơn', 'Xã Ia Mrơn'),
    ('Tỉnh Gia Lai', "Xã K'Dang", 'Xã KDang'),
    ('Tỉnh Gia Lai', "Xã H'Neng", 'Xã Hneng'),
    ('Tỉnh Đắk Lắk', "Xã Ea H'đinh", "Xã Ea H'đing"),
    ('Tỉnh Đắk Lắk', 'Xã Cư K Róa', 'Xã Cư Króa'),
    ('Tỉnh Đắk Lắk', 'Xã Cư Jang', 'Xã Cư Yang'),
    ('Tỉnh Đắk Lắk', 'Xã EaBia', 'Xã Ea Bia'),
    ('Tỉnh Đắk Lắk', 'Xã Ealy', 'Xã Ea Ly'),
    ('Tỉnh Đắk Lắk', 'Xã Cư Ê Wi', 'Xã Cư Êwi'),
    ('Tỉnh Đắk Lắk', "Xã Ea R'Bin", 'Xã Ea Rbin'),
    ('Tỉnh Đắk Lắk', 'Xã EaTrol', 'Xã Ea Trol'),
    ('Tỉnh Đắk Lắk', 'Xã Eachà Rang', 'Xã Ea Chà Rang'),
    ('Tỉnh Lào Cai', 'Thị trấn N.T Phong Hải', 'Thị trấn Nông trường Phong Hải'),
    ('Tỉnh Lào Cai', 'Xã Dế Su Phình', 'Xã Dế Xu Phình'),
    ('Tỉnh Lào Cai', 'Xã Nàn Xín', 'Xã Nàn Sín'),
    ('Tỉnh Lào Cai', 'Thị trấn NT Trần Phú', 'Thị trấn Nông trường Trần Phú'),
    ('Tỉnh Lâm Đồng', 'Xã Đạm Bri', "Xã Đamb'ri"),
    ('Tỉnh Lâm Đồng', "Xã Đắk N'Dung", "Xã Đắk N'Drung"),
    ('Tỉnh Lâm Đồng', "Xã Đắk DRông", "Xã Đắk D'rông"),
    ('Tỉnh Lâm Đồng', 'Xã Thuận Quí', 'Xã Thuận Quý'),
    ('Thành phố Hồ Chí Minh', 'Xã Quy Đức', 'Xã Qui Đức'),
    ('Tỉnh Vĩnh Long', 'Xã An Quy', 'Xã An Qui'),
    ('Tỉnh Vĩnh Long', 'Xã Lục Sỹ Thành', 'Xã Lục Sĩ Thành'),
    ('Tỉnh An Giang', 'Xã Ô Long Vỹ', 'Xã Ô Long Vĩ'),
    ('Thành phố Cần Thơ', 'Xã Hòa Tú II', 'Xã Hòa Tú 2'),
    ('Tỉnh Đắk Lắk', 'Xã EaBar', 'Xã Ea Bar'),
    ('Tỉnh Đắk Lắk', "Xã Ea D'rơng", 'Xã Ea Drơng'),
    ('Thủ đô Hà Nội', 'Phường Phương Liên - Trung Tự', 'Phường Phương Liên Trung Tự'),
    ('Thủ đô Hà Nội', 'Phường Văn Miếu - Quốc Tử Giám', 'Phường Văn Miếu Quốc Tử Giám'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Rơ Ông', 'Xã Đăk Rơ Ông'),
    ('Tỉnh Cao Bằng', 'Xã Phong Nậm', 'Xã Phong Nặm'),
    ('Tỉnh Tuyên Quang', 'Xã Nậm Tỵ', 'Xã Nậm Ty'),
    ('Tỉnh Tuyên Quang', 'Thị trấn Phó Bảng', 'Thị trấn Phố Bảng'),
    ('Tỉnh Tuyên Quang', 'Xã Khau Tinh', 'Xã Khâu Tinh'),
    ('Tỉnh Lào Cai', 'Thị trấn Mù Căng Chải', 'Thị trấn Mù Cang Chải'),
    ('Tỉnh Lào Cai', 'Xã Nậm Chầy', 'Xã Nậm Chày'),
    ('Tỉnh Lào Cai', 'Xã Tả Ngải Chồ', 'Xã Tả Ngài Chồ'),
    ('Tỉnh Thái Nguyên', 'Phường Quán Triều', 'Phường Quan Triều'),
    ('Tỉnh Phú Thọ', 'Xã Mi Hòa', 'Xã Mỵ Hòa'),
    ('Tỉnh Điện Biên', 'Xã Luân Giới', 'Xã Luân Giói'),
    ('Tỉnh Điện Biên', 'Xã Pú Nhi', 'Xã Pu Nhi'),
    ('Tỉnh Điện Biên', 'Xã Noọng Hẹt', 'Xã Noong Hẹt'),
    ('Tỉnh Lai Châu', 'Xã Pa Vệ Sử', 'Xã Pa Vệ Sủ'),
    ('Tỉnh Lai Châu', 'Xã Tà Gia', 'Xã Ta Gia'),
    ('Tỉnh Sơn La', 'Xã Nậm Ét', 'Xã Nặm Ét'),
    ('Tỉnh Sơn La' , 'Xã Nà Ơt', 'Xã Nà Ớt'),
    ('Tỉnh Thanh Hóa', 'Xã Vân Âm', 'Xã Vân Am'),
    ('Tỉnh Thanh Hóa', 'Xã Văn Sơn', 'Xã Vân Sơn'),
    ('Tỉnh Nghệ An', 'Xã Lưỡng Minh', 'Xã Lượng Minh'),
    ('Tỉnh Nghệ An', 'Xã Căm Muộn', 'Xã Cắm Muộn'),
    ('Tỉnh Hà Tĩnh', 'Xã Cỗ Đạm', 'Xã Cổ Đạm'),
    ('Thành phố Đà Nẵng', 'Xã Ka Dăng', 'Xã Kà Dăng'),
    ('Thành phố Đà Nẵng', 'Xã Za Hung', 'Xã Zà Hung'),
    ('Tỉnh Quảng Ngãi', 'Xã Ngok Tem', 'Xã Ngọc Tem'),
    ('Tỉnh Quảng Ngãi', 'Xã Ngok Wang', 'Xã Ngọk Wang'),
    ('Tỉnh Quảng Ngãi', 'Xã Ngok Réo', 'Xã Ngọk Réo'),
    ('Tỉnh Quảng Ngãi', 'Xã Rơ Kơi', 'Xã Rờ Kơi'),
    ('Tỉnh Gia Lai', 'Xã Chư Đăng Ya', 'Xã Chư Đang Ya'),
    ('Tỉnh Gia Lai', 'Xã Chư Krêy', 'Xã Chư Krey'),
    ('Tỉnh Gia Lai', 'Thị trấn Đăk Đoa', 'Thị trấn Đak Đoa'),
    ('Tỉnh Gia Lai', 'Xã Đăk Sơmei', 'Xã Đak Sơmei'),
    ('Tỉnh Gia Lai', 'Xã Ia KRai', 'Xã Ia Krăi'),
    ('Tỉnh Gia Lai', 'Xã Đăk Krong', 'Xã Đak Krong'),
    ('Tỉnh Gia Lai', 'Xã Đăk Djrăng', 'Xã Đak Djrăng'),
    ('Tỉnh Gia Lai', 'Xã Ia Mláh', 'Xã Ia Mlăh'),
    ('Tỉnh Đắk Lắk', 'Xã Cuor Đăng', 'Xã Cuôr Đăng'),
    ('Tỉnh Đắk Lắk', 'Xã ĐLiê Ya', 'Xã Dliê Ya'),
    ('Tỉnh Đắk Lắk', 'Xã Ea KNuec', 'Xã Ea Knuếc'),
    ('Tỉnh Đắk Lắk', "Xã Ea M'DRóh", "Xã Ea M'Droh"),
    ('Tỉnh Đắk Lắk', "Xã Chư KBô", "Xã Chứ Kbô"),
    ('Tỉnh Đắk Lắk', "Thị trấn M'Đrắk", "Thị trấn M'Drắk"),
    ('Tỉnh Đắk Lắk', "Xã Ea Puk", "Xã Ea Púk"),
    ('Tỉnh Lâm Đồng', "Xã Đạ M'Rong", "Xã Đạ M'Rông"),
    ('Tỉnh Lâm Đồng', "Xã Buôn Choah", "Xã Buôn Choáh"),
    ('Tỉnh Lâm Đồng', "Xã Đăk DRông", "Xã Đăk D'rông"),
    ('Tỉnh Đồng Nai', "Xã Dak Lua", "Xã Đắc Lua"),
    ('Tỉnh Vĩnh Long', "Xã Qưới Sơn", "Xã Quới Sơn"),
    ('Tỉnh Vĩnh Long', "Xã Vĩnh Kim", "Xã Vinh Kim"),
    ('Tỉnh An Giang', "Phường Vĩnh Ngươn", "Phường Vĩnh Nguơn"),
    ('Thành phố Cần Thơ', "Thị trấn Thanh An", "Thị trấn Thạnh An"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Xú", "Xã Đăk Xú"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Cấm", "Xã Đăk Cấm"),
    ('Tỉnh Quảng Ngãi', "Thị trấn Đắk Hà", "Thị trấn Đăk Hà"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk La", "Xã Đăk La"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Tơ Lung", "Xã Đăk Tơ Lung"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Kôi", "Xã Đăk Kôi"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Hring", "Xã Đăk Hring"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Mar", "Xã Đăk Mar"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Kroong", "Xã Đăk Kroong"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Môn", "Xã Đăk Môn"),
    ('Tỉnh Quảng Ngãi', "Thị trấn Đắk Glei", "Thị trấn Đăk Glei"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Pék", "Xã Đăk Pék"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Nhoong", "Xã Đăk Nhoong"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Man", "Xã Đăk Man"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Blô", "Xã Đăk Blô"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Pxi", "Xã Đăk Pxi"),
    ('Tỉnh Quảng Ngãi', "Xã Đắk Blà", "Xã Đăk Blà"),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Rơ Wa', 'Xã Đăk Rơ Wa'),
    ('Tỉnh Quảng Ngãi', 'Thị trấn Đắk Rve', 'Thị trấn Đăk Rve'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Pne', 'Xã Đăk Pne'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Na', 'Xã Đăk Na'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Sao', 'Xã Đăk Sao'),
    ('Tỉnh Quảng Ngãi', 'Thị trấn Đắk Tô', 'Thị trấn Đăk Tô'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Tờ Kan', 'Xã Đăk Tờ Kan'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Ui', 'Xã Đăk Ui'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Ang', 'Xã Đăk Ang'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Dục', 'Xã Đăk Dục'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Nông', 'Xã Đăk Nông'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Tờ Re', 'Xã Đăk Tờ Re'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Ruồng', 'Xã Đăk Ruồng'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Trăm', 'Xã Đăk Trăm'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Nên', 'Xã Đăk Nên'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Ring', 'Xã Đăk Ring'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Tăng', 'Xã Đăk Tăng'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Rơ Nga', 'Xã Đăk Rơ Nga'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Kan', 'Xã Đăk Kan'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Hà', 'Xã Đăk Hà'),
    ('Tỉnh Quảng Ngãi', 'Xã Đắk Choong', 'Xã Đăk Choong'),
    ('Tỉnh Gia Lai', 'Xã Đắk Kơ Ning', 'Xã Đăk Kơ Ning'),
    ('Tỉnh Lâm Đồng', "Xã Đăk R'Moan", "Xã Đắk R'Moan"),
    ('Thành phố Hải Phòng', 'Xã Bình Lăng', 'Xã Bình Lãng'),

]

spelling_sapnhap = [
    # newProvince, ward, correct ward
    ('Tỉnh Ninh Bình', 'Phường Tiên Sơn', 'Xã Tiên Sơn'),
    ('Tỉnh Thái Nguyên', 'Xã Đắc Sơn', 'Phường Đắc Sơn'),
    ('Tỉnh Lai Châu', 'Xã Can Hồ', 'Xã Kan Hồ'),
    ('Tỉnh Sơn La', 'Phường Thảo Ngu Yên', 'Phường Thảo Nguyên'),
    ('Tỉnh Sơn La', 'Xã Xí M Vàng', 'Xã Xím Vàng'),
    ('Tỉnh Nghệ An', 'Xã Thu Thủy', 'Phường Thu Thủy'),
    ('Thành phố Huế', 'Phường Đúc', 'Phường Phường Đúc'),
    ('Thành phố Đà Nẵng', 'Xã Bhalêê', 'Xã Bha Lêê'),
    ('Thành phố Đà Nẵng', 'Xã Avương', 'Xã A Vương'),
    ('Thành phố Đà Nẵng', 'Xã Gari', 'Xã Ga Ri'),
    ('Thành phố Đà Nẵng', 'Xã Axan', 'Xã A Xan'),
    ('Thành phố Đà Nẵng', 'Xã Atiêng', 'Xã A Tiêng'),
    ('Thành phố Đà Nẵng', 'Xã Anông', 'Xã A Nông'),
    ('Tỉnh Quảng Ngãi', 'Lê Lợi', 'Phường Lê Lợi'),
    ('Tỉnh Quảng Ngãi', 'Nguyễn Trãi', 'Phường Nguyễn Trãi'),
    ('Tỉnh Quảng Ngãi', 'Xã Đăk Plô', 'Xã Đăk Blô'),
    ('Tỉnh Quảng Ngãi', 'Phổ Hòa', 'Phường Phổ Hòa'),
    ('Tỉnh Quảng Ngãi', 'Phổ Minh', 'Phường Phổ Minh'),
    ('Tỉnh Quảng Ngãi', 'Phổ Vinh', 'Phường Phổ Vinh'),
    ('Tỉnh Quảng Ngãi', 'Phổ Ninh', 'Phường Phổ Ninh'),
    ('Tỉnh Gia Lai', 'Xã Chơ GLong', 'Xã Chơ Long'),
    ('Tỉnh Gia Lai', 'Xã Adơk', 'Xã A Dơk'),
    ('Tỉnh Gia Lai', 'Xã Ia Dreh', 'Ia HDreh'),
    ('Tỉnh Đắk Lắk', 'Krông Na', 'Xã Krông Na'),
    ('Tỉnh Đắk Lắk', 'Xã Cư Dliê Mnông', "Xã Cư Dliê M'nông"),
    ('Tỉnh Lâm Đồng', 'Xã ĐamBri', "Xã Đamb'ri"),
    ('Tỉnh Lâm Đồng', "Xã Cư K'nia", 'Xã Cư Knia'),
    ('Tỉnh Lâm Đồng', 'Thị trấn Mađaguôi', 'Thị trấn Ma Đa Guôi'),
    ('Tỉnh Lâm Đồng', 'Xã Mađaguôi', 'Xã Ma Đa Guôi'),
    ('Tỉnh Lào Cai', 'Xã Tà Xi Láng', 'Xã Tà Si Láng'),
    ('Tỉnh Gia Lai', 'Ia HDreh', 'Xã Ia HDreh'),
    ('Tỉnh Lâm Đồng', "Xã Đưng K'Nớ", 'Xã Đưng KNớ'),
    ('Tỉnh Lâm Đồng', "Xã N'Thôn Hạ", "Xã N'Thol Hạ"),
    ('Tỉnh Tây Ninh', 'Thủ Thừa', 'Thị trấn Thủ Thừa'),
    ('Tỉnh Tây Ninh', 'Phường IV', 'Phường 4'),
    ('Tỉnh Đồng Nai', 'Xã Xuân An', 'Phường Xuân An'),
    ('Tỉnh Đồng Nai', 'Xã Xuân Bình', 'Phường Xuân Bình'),
    ('Thành phố Hồ Chí Minh', 'Rạch Ông', 'Phường Rạch Ông'),
    ('Tỉnh Đồng Tháp', 'Xã Mỹ Quí', 'Xã Mỹ Quý'),
    ('Thành phố Cần Thơ', 'Xã Tân Lộc', 'Phường Tân Lộc'),
    ('Tỉnh Lào Cai', 'Xã Thẳm Dương', 'Xã Thẩm Dương'),
    ('Tỉnh Sơn La', 'Xã Nặm Păm', 'Xã Nậm Păm'),
    ('Tỉnh Quảng Ngãi', 'Xã Ngọk Tem', 'Xã Ngọc Tem'),
    ('Tỉnh Gia Lai', 'Xã Ia Drang', 'Xã Ia Drăng'),
    ('Tỉnh Gia Lai', 'Xã Đak Rong', 'Xã Đăk Rong'),
    ('Tỉnh Gia Lai', 'Xã Ia Krái', 'Xã Ia Krăi'),
    ('Tỉnh Gia Lai', 'Xã Ia Trôk', 'Xã Ia Trok'),
    ('Tỉnh Gia Lai', 'Xã Đak Trôi', 'Xã Đăk Trôi'),
    ('Tỉnh Gia Lai', 'Xã Ia Mlah', 'Xã Ia Mlăh'),
    ('Tỉnh Đắk Lắk', 'Xã Ea Khăl', 'Xã Ea Khal'),
    ('Tỉnh Đồng Nai', "Xã Đak Lua", "Xã Đắc Lua"),
    ('Tỉnh Đồng Nai', "Xã Đăk Ơ", "Xã Đak Ơ"),

]

# Nhận thấy các lỗi sai thường là: qui - quy, s - x, \bch - \btr, nt - nong truong, 2 - II, 4 - IV (Số la mã), ngok - ngoc. Sau này sẽ tạo key cho tính năng parse address

for s in spelling_danhmuc:
    newProvince, ward, WardCorrect = s
    df_danhmuc.loc[(df_danhmuc['newProvince']==newProvince) & (df_danhmuc['ward'].fillna('').str.lower()==ward.lower()), 'ward'] = WardCorrect

for s in spelling_sapnhap:
    newProvince, ward, WardCorrect = s
    df_sapnhap_ward.loc[(df_sapnhap_ward['newProvince']==newProvince) & (df_sapnhap_ward['ward'].fillna('').str.lower()==ward.lower()), 'ward'] = WardCorrect

In [26]:
# Case đặc biệt: Sau khi merge xong mới kiểm tra ra, phải quay lại bổ sung ở bước này
df_sapnhap_ward.loc[(df_sapnhap_ward['newProvince']=='Tỉnh Đồng Nai') & (df_sapnhap_ward['newWard']=='Long Khánh') & (df_sapnhap_ward['ward']=='Xã Xuân Hòa'), 'ward'] = 'Phường Xuân Hòa'
df_sapnhap_ward.loc[(df_sapnhap_ward['newProvince']=='Tỉnh Đồng Nai') & (df_sapnhap_ward['newWard']=='Long Khánh') & (df_sapnhap_ward['ward']=='Xã Phú Bình'), 'ward'] = 'Phường Phú Bình'
df_danhmuc.loc[(df_danhmuc['province']=='Tỉnh Kon Tum') & (df_danhmuc['district']=='Huyện Đắk Glei') & (df_danhmuc['ward']=='Xã Đắk Long'), 'ward'] = 'Xã Đăk Long'

In [27]:
# Tạo các cột để tiện làm việc

def create_format_cols():
    for col in ['district', 'ward']:
        # Unidecode để tiện filter và kiểm tra
        df_danhmuc[f'{col}Ud'] = df_danhmuc[col].apply(unidecode_pro)
        df_sapnhap_ward[f'{col}Ud'] = df_sapnhap_ward[col].apply(unidecode_pro)

        # Lower giúp merge dễ hơn nhưng an toàn hơn unidecode. Vì có có những ward chỉ khác dấu tiếng Việt trong cùng một province hay một district
        df_danhmuc[f'{col}Lower'] = df_danhmuc[col].apply(lower_safely)
        df_sapnhap_ward[f'{col}Lower'] = df_sapnhap_ward[col].apply(lower_safely)


create_format_cols()

In [28]:
# Tạo bộ key để kiểm tra mismatched

# Case 1: Có newProvince, province, district, ward. Đã biết chỉ có 2 province.
key1 = 'keyNewProvinceProvinceDistrictWard'

# Case 2: Có newProvince, district, ward.
# Có sai nhiều do fill_district không phải luôn đúng, Vậy nên:
# - Kiểm tra mismached: Chỉ kiểm tra mismatched cho cho key3 và key4, không kiểm tra key2.
# - Merge: Một phần key2 sẽ match, phần còn lại sẽ bỏ district và đưa vào key4
key2 = 'keyNewProvinceDistrictWard'

# Case 3: Có newProvince, district.
key3 = 'keyNewProvinceDistrict'

# Case 4: Có newProvince, ward.
key4 = 'keyNewProvinceWard'

def create_key_cols(for_merging=False):
    '''
    Create a function for reusable after fixed mismatched ward and before merging
    '''
    # Case 1
    for df in [df_sapnhap_ward, df_danhmuc]:
        df[key1] = np.where((~df['province'].isna()) & (~df['district'].isna()) & (~df['ward'].isna()), df['newProvince'] + '_' + df['province'] + '_' + df['districtLower'] + '_' + df['wardLower'], np.nan)

    # Case 2
    ## danhmuc
    df_danhmuc[key2] = np.where((~df_danhmuc['district'].isna()) & (~df_danhmuc['ward'].isna()), df_danhmuc['newProvince'] + '_' + df_danhmuc['districtLower'] + '_' + df_danhmuc['wardLower'], np.nan)

    ## sapnhap
    if for_merging:
        df_sapnhap_ward[key2] = np.where((~df_sapnhap_ward['district'].isna()) & (~df_sapnhap_ward['ward'].isna())
                                         & (df_sapnhap_ward['province'].isna()), # Thêm province là nan
                                         df_sapnhap_ward['newProvince'] + '_' + df_sapnhap_ward['districtLower'] + '_' + df_sapnhap_ward['wardLower'], np.nan)
    else:
        df_sapnhap_ward[key2] = np.where((~df_sapnhap_ward['district'].isna()) & (~df_sapnhap_ward['ward'].isna()),
                                         df_sapnhap_ward['newProvince'] + '_' + df_sapnhap_ward['districtLower'] + '_' + df_sapnhap_ward['wardLower'], np.nan)

    # Case 3
    ## danh muc
    df_danhmuc[key3] = np.where((~df_danhmuc['district'].isna()), df_danhmuc['newProvince'] + '_' + df_danhmuc['districtLower'], np.nan)

    ## sapnhap
    if for_merging:
        df_sapnhap_ward[key3] = np.where((~df_sapnhap_ward['district'].isna())
                                         & (df_sapnhap_ward['ward'].isna()), # Thêm ward là nan
                                         df_sapnhap_ward['newProvince'] + '_' + df_sapnhap_ward['districtLower'], np.nan)
    else:
        df_sapnhap_ward[key3] = np.where((~df_sapnhap_ward['district'].isna()),
                                         df_sapnhap_ward['newProvince'] + '_' + df_sapnhap_ward['districtLower'], np.nan)

    ## Case 4
    ## danhmuc
    df_danhmuc[key4] = np.where((~df_danhmuc['ward'].isna()), df_danhmuc['newProvince'] + '_' + df_danhmuc['wardLower'], np.nan)

    ## sapnhap
    if for_merging:
        df_sapnhap_ward[key4] = np.where((~df_sapnhap_ward['ward'].isna())
                                         & (df_sapnhap_ward['district'].isna()), # Thêm district là nan
                                         df_sapnhap_ward['newProvince'] + '_' + df_sapnhap_ward['wardLower'], np.nan)
    else:
        df_sapnhap_ward[key4] = np.where((~df_sapnhap_ward['ward'].isna()),
                                         df_sapnhap_ward['newProvince'] + '_' + df_sapnhap_ward['wardLower'], np.nan)

# Thực hiện tạo key
# create_key_cols()

In [29]:
def create_key_cols(for_merging=False):
    """
    Tạo các cột key để phục vụ kiểm tra mismatched và merge dữ liệu.
    Gồm 4 case: key1, key2, key3, key4 với từng điều kiện khác nhau.
    """

    def gen_key(df, condition, columns, key_name):
        df[key_name] = np.where(condition, df[columns[0]], np.nan)
        for col in columns[1:]:
            df[key_name] = np.where(condition, df[key_name] + '_' + df[col], df[key_name])

    # --- Các key cần tạo ---
    keys = {
        key1: {
            'condition': lambda df: df['province'].notna() & df['district'].notna() & df['ward'].notna(),
            'columns': ['newProvince', 'province', 'districtLower', 'wardLower']
        },
        key2: {
            'condition': lambda df: df['district'].notna() & df['ward'].notna(),
            'columns': ['newProvince', 'districtLower', 'wardLower']
        },
        key3: {
            'condition': lambda df: df['district'].notna(),
            'columns': ['newProvince', 'districtLower']
        },
        key4: {
            'condition': lambda df: df['ward'].notna(),
            'columns': ['newProvince', 'wardLower']
        },
    }

    # --- df_danhmuc ---
    for key, conf in keys.items():
        gen_key(df_danhmuc, conf['condition'](df_danhmuc), conf['columns'], key)

    # --- df_sapnhap_ward ---
    for key, conf in keys.items():
        cond = conf['condition'](df_sapnhap_ward)

        # Xử lý riêng một số case trong chế độ for_merging
        if for_merging:
            if key == key2:
                cond = cond & df_sapnhap_ward['province'].isna()
            elif key == key3:
                cond = cond & df_sapnhap_ward['ward'].isna()
            elif key == key4:
                cond = cond & df_sapnhap_ward['district'].isna()

        gen_key(df_sapnhap_ward, cond, conf['columns'], key)


create_key_cols()

In [30]:
# Preview df_sapnhap_ward (2 province duy nhất trong dataset này)
df_sapnhap_ward[~df_sapnhap_ward[key1].isna()]

,province,district,ward,isMergedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
9222,Tỉnh Trà Vinh,Huyện Châu Thành,Thị trấn Châu Thành,True,Châu Thành,Tỉnh Vĩnh Long,2880,xã,106.337,9.83937,47.71,Thị trấn Châu Thành (Huyện Châu Thành - Tỉn...,huyen chau thanh,huyện châu thành,thi tran chau thanh,thị trấn châu thành,Tỉnh Vĩnh Long_Tỉnh Trà Vinh_huyện châu thành_...,Tỉnh Vĩnh Long_huyện châu thành_thị trấn châu ...,Tỉnh Vĩnh Long_huyện châu thành,Tỉnh Vĩnh Long_thị trấn châu thành
9374,Tỉnh Bến Tre,Huyện Châu Thành,Thị trấn Châu Thành,True,Phú Túc,Tỉnh Vĩnh Long,2934,xã,106.318,10.30100,63.14,Thị trấn Châu Thành (Huyện Châu Thành - Tỉn...,huyen chau thanh,huyện châu thành,thi tran chau thanh,thị trấn châu thành,Tỉnh Vĩnh Long_Tỉnh Bến Tre_huyện châu thành_t...,Tỉnh Vĩnh Long_huyện châu thành_thị trấn châu ...,Tỉnh Vĩnh Long_huyện châu thành,Tỉnh Vĩnh Long_thị trấn châu thành


In [31]:
# Preview df_danhmuc (Các đảo thì không có ward)
df_danhmuc[df_danhmuc[key1].isna()]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,...,newProvinceLon,isMergedProvince,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
3582,318,31,Huyện,Huyện Bạch Long Vĩ,Thành phố Hải Phòng,NaN,NaN,NaN,4,Thành phố Hải Phòng,...,106.470,True,huyen bach long vi,huyện bạch long vĩ,NaN,NaN,NaN,NaN,Thành phố Hải Phòng_huyện bạch long vĩ,NaN
5794,471,45,Huyện,Huyện Cồn Cỏ,Tỉnh Quảng Trị,NaN,NaN,NaN,19,Tỉnh Quảng Trị,...,106.527,True,huyen con co,huyện cồn cỏ,NaN,NaN,NaN,NaN,Tỉnh Quảng Trị_huyện cồn cỏ,NaN
5975,498,48,Huyện,Huyện Hoàng Sa,Thành phố Đà Nẵng,NaN,NaN,NaN,21,Thành phố Đà Nẵng,...,107.966,True,huyen hoang sa,huyện hoàng sa,NaN,NaN,NaN,NaN,Thành phố Đà Nẵng_huyện hoàng sa,NaN
6379,536,51,Huyện,Huyện Lý Sơn,Tỉnh Quảng Ngãi,NaN,NaN,NaN,22,Tỉnh Quảng Ngãi,...,108.143,True,huyen ly son,huyện lý sơn,NaN,NaN,NaN,NaN,Tỉnh Quảng Ngãi_huyện lý sơn,NaN
8196,755,77,Huyện,Huyện Côn Đảo,Tỉnh Bà Rịa - Vũng Tàu,NaN,NaN,NaN,29,Thành phố Hồ Chí Minh,...,106.638,True,huyen con dao,huyện côn đảo,NaN,NaN,NaN,NaN,Thành phố Hồ Chí Minh_huyện côn đảo,NaN


In [32]:
# Check Key 1
df_sapnhap_ward[~df_sapnhap_ward[key1].isin(df_danhmuc[key1])]
# Key 1 is valid fully

,province,district,ward,isMergedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard


In [33]:
# Check Key 3
df_sapnhap_ward[~df_sapnhap_ward[key3].isin(df_danhmuc[key3]) & (~df_sapnhap_ward[key3].isna())][['newProvince', 'district', 'districtUd']].drop_duplicates()

,newProvince,district,districtUd
6668,Tỉnh Quảng Ngãi,Huyện Đăk Hà,huyen dak ha
7250,Tỉnh Gia Lai,Huyện Đak Đoa,huyen dak doa


In [34]:
# Chổ để tìm ward bị mismatch trong df_danhmuc
condition1 = (df_danhmuc['newProvince'] == 'Tỉnh Gia Lai')
condition2 = (df_danhmuc['districtUd'].fillna('').str.contains('dak doa'))
df_danhmuc[condition1 & condition2][['newProvince', 'province', 'district', 'districtUd']].drop_duplicates()

,newProvince,province,district,districtUd
7111,Tỉnh Gia Lai,Tỉnh Gia Lai,Huyện Đăk Đoa,huyen dak doa


In [35]:
# Fix case 3 trực tiếp tại đây
df_danhmuc.loc[(df_danhmuc['province']=='Tỉnh Kon Tum') & (df_danhmuc['district']=='Huyện Đắk Hà'), 'district'] = 'Huyện Đăk Hà'
df_danhmuc.loc[(df_danhmuc['province']=='Tỉnh Gia Lai') & (df_danhmuc['district']=='Huyện Đăk Đoa'), 'district'] = 'Huyện Đak Đoa'

In [36]:
# Tạo lại các cột đẻ check lại key 3
create_format_cols()
create_key_cols()

In [37]:
# Check key 4
df_sapnhap_ward[~df_sapnhap_ward[key4].isin(df_danhmuc[key4]) & (~df_sapnhap_ward[key4].isna())][['newProvince', 'district', 'ward', 'wardUd', 'isMergedWard']].drop_duplicates()
# Chỉ còn "Xã Tân Phước" là không tồn tại ở Tỉnh Tuyên Quang và Tỉnh Hà Giang. Có lẽ dữ liệu của sapnhap bị sai chổ này

,newProvince,district,ward,wardUd,isMergedWard
2738,Tỉnh Tuyên Quang,NaN,Xã Tân Phước,xa tan phuoc,True


In [38]:
# Chổ để tìm ward bị mismatch trong df_danhmuc
condition1 = (df_danhmuc['newProvince'] == 'Thành phố Hải Phòng')
condition2 = (df_danhmuc['wardUd'].fillna('').str.contains('binh lang'))
df_danhmuc[condition1 & condition2][['newProvince', 'province', 'district', 'ward', 'wardUd']].drop_duplicates()

,newProvince,province,district,ward,wardUd
3365,Thành phố Hải Phòng,Tỉnh Hải Dương,Huyện Tứ Kỳ,Xã Bình Lãng,xa binh lang


## Kiểm tra trước khi merge

In [39]:
# Thực hiện lại việc tạo key cho đã sửa chỉnh tả
create_format_cols()
create_key_cols(for_merging=True)

In [40]:
# Case 1
sapnhap1 = df_sapnhap_ward[~df_sapnhap_ward[key1].isna()]
sapnhap1

,province,district,ward,isMergedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
9222,Tỉnh Trà Vinh,Huyện Châu Thành,Thị trấn Châu Thành,True,Châu Thành,Tỉnh Vĩnh Long,2880,xã,106.337,9.83937,47.71,Thị trấn Châu Thành (Huyện Châu Thành - Tỉn...,huyen chau thanh,huyện châu thành,thi tran chau thanh,thị trấn châu thành,Tỉnh Vĩnh Long_Tỉnh Trà Vinh_huyện châu thành_...,NaN,NaN,NaN
9374,Tỉnh Bến Tre,Huyện Châu Thành,Thị trấn Châu Thành,True,Phú Túc,Tỉnh Vĩnh Long,2934,xã,106.318,10.30100,63.14,Thị trấn Châu Thành (Huyện Châu Thành - Tỉn...,huyen chau thanh,huyện châu thành,thi tran chau thanh,thị trấn châu thành,Tỉnh Vĩnh Long_Tỉnh Bến Tre_huyện châu thành_t...,NaN,NaN,NaN


In [41]:
# Case 2: Chỉ lấy phần có match, phần còn lại sẽ đưa vào Case 4b
sapnhap2 = df_sapnhap_ward[(~df_sapnhap_ward[key2].isna()) & (df_sapnhap_ward[key2].isin(df_danhmuc[key2]))]
sapnhap2

,province,district,ward,isMergedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
20,NaN,Quận Hai Bà Trưng,Phường Bạch Mai,True,Bạch Mai,Thủ đô Hà Nội,5,phường,105.852,21.00220,2.95,Phường Bạch Mai (quận Hai Bà Trưng),quan hai ba trung,quận hai bà trưng,phuong bach mai,phường bạch mai,NaN,Thủ đô Hà Nội_quận hai bà trưng_phường bạch mai,NaN,NaN
21,NaN,Quận Hai Bà Trưng,Phường Bách Khoa,True,Bạch Mai,Thủ đô Hà Nội,5,phường,105.852,21.00220,2.95,Phường Bách Khoa (quận Hai Bà Trưng),quan hai ba trung,quận hai bà trưng,phuong bach khoa,phường bách khoa,NaN,Thủ đô Hà Nội_quận hai bà trưng_phường bách khoa,NaN,NaN
22,NaN,Quận Hai Bà Trưng,Phường Quỳnh Mai,True,Bạch Mai,Thủ đô Hà Nội,5,phường,105.852,21.00220,2.95,Phường Quỳnh Mai (quận Hai Bà Trưng),quan hai ba trung,quận hai bà trưng,phuong quynh mai,phường quỳnh mai,NaN,Thủ đô Hà Nội_quận hai bà trưng_phường quỳnh mai,NaN,NaN
23,NaN,Quận Hai Bà Trưng,Phường Minh Khai,True,Bạch Mai,Thủ đô Hà Nội,5,phường,105.852,21.00220,2.95,Phường Minh Khai (quận Hai Bà Trưng),quan hai ba trung,quận hai bà trưng,phuong minh khai,phường minh khai,NaN,Thủ đô Hà Nội_quận hai bà trưng_phường minh khai,NaN,NaN
75,NaN,Huyện Phú Xuyên,Xã Tân Dân,True,Chuyên Mỹ,Thủ đô Hà Nội,13,xã,105.884,20.71620,35.54,Xã Tân Dân (huyện Phú Xuyên),huyen phu xuyen,huyện phú xuyên,xa tan dan,xã tân dân,NaN,Thủ đô Hà Nội_huyện phú xuyên_xã tân dân,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10521,NaN,Thành phố Cà Mau,Phường 5,True,Tân Thành,Tỉnh Cà Mau,3340,phường,105.217,9.18698,48.42,Phường 5 (thành phố Cà Mau),thanh pho ca mau,thành phố cà mau,phuong 5,phường 5,NaN,Tỉnh Cà Mau_thành phố cà mau_phường 5,NaN,NaN
10522,NaN,Thành phố Cà Mau,Phường Tân Thành,True,Tân Thành,Tỉnh Cà Mau,3340,phường,105.217,9.18698,48.42,Phường Tân Thành (thành phố Cà Mau),thanh pho ca mau,thành phố cà mau,phuong tan thanh,phường tân thành,NaN,Tỉnh Cà Mau_thành phố cà mau_phường tân thành,NaN,NaN
10523,NaN,Thành phố Cà Mau,Xã Tân Thành,True,Tân Thành,Tỉnh Cà Mau,3340,phường,105.217,9.18698,48.42,Xã Tân Thành (thành phố Cà Mau),thanh pho ca mau,thành phố cà mau,xa tan thanh,xã tân thành,NaN,Tỉnh Cà Mau_thành phố cà mau_xã tân thành,NaN,NaN
10524,NaN,Thành phố Cà Mau,Phường 7,True,Tân Thành,Tỉnh Cà Mau,3340,phường,105.217,9.18698,48.42,Phường 7 (thành phố Cà Mau),thanh pho ca mau,thành phố cà mau,phuong 7,phường 7,NaN,Tỉnh Cà Mau_thành phố cà mau_phường 7,NaN,NaN


In [42]:
# Case 3
sapnhap3 = df_sapnhap_ward[~df_sapnhap_ward[key3].isna()]
sapnhap3

,province,district,ward,isMergedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
1349,NaN,Huyện Bạch Long Vĩ,NaN,True,Bạch Long Vĩ,Thành phố Hải Phòng,1050,đặc khu,107.7307,20.13460,3.07,Huyện Bạch Long Vĩ,huyen bach long vi,huyện bạch long vĩ,NaN,NaN,NaN,NaN,Thành phố Hải Phòng_huyện bạch long vĩ,NaN
5960,NaN,Huyện Cồn Cỏ,NaN,True,Cồn Cỏ,Tỉnh Quảng Trị,1764,đặc khu,107.3390,17.15925,2.30,Huyện Cồn Cỏ,huyen con co,huyện cồn cỏ,NaN,NaN,NaN,NaN,Tỉnh Quảng Trị_huyện cồn cỏ,NaN
6415,NaN,Huyện Hoàng Sa,NaN,True,Hoàng Sa,Thành phố Đà Nẵng,1904,đặc khu,112.1350,16.56800,305.00,Huyện Hoàng Sa,huyen hoang sa,huyện hoàng sa,NaN,NaN,NaN,NaN,Thành phố Đà Nẵng_huyện hoàng sa,NaN
6734,NaN,Huyện Lý Sơn,NaN,True,Lý Sơn,Tỉnh Quảng Ngãi,2015,đặc khu,109.1150,15.38500,10.39,Huyện Lý Sơn,huyen ly son,huyện lý sơn,NaN,NaN,NaN,NaN,Tỉnh Quảng Ngãi_huyện lý sơn,NaN
8807,NaN,Huyện Côn Đảo,NaN,True,Côn Đảo,Thành phố Hồ Chí Minh,2741,đặc khu,106.6115,8.69810,75.79,Huyện Côn Đảo,huyen con dao,huyện côn đảo,NaN,NaN,NaN,NaN,Thành phố Hồ Chí Minh_huyện côn đảo,NaN
9931,NaN,Huyện Kiên Hải,NaN,True,Kiên Hải,Tỉnh An Giang,3125,đặc khu,104.6320,9.80000,21.81,Huyện Kiên Hải,huyen kien hai,huyện kiên hải,NaN,NaN,NaN,NaN,Tỉnh An Giang_huyện kiên hải,NaN


In [43]:
# Case 4a
sapnhap4a = df_sapnhap_ward[~df_sapnhap_ward[key4].isna()]
sapnhap4a

,province,district,ward,isMergedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
0,NaN,NaN,Xã Đông La,True,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Xã Đông La,NaN,NaN,xa dong la,xã đông la,NaN,NaN,NaN,Thủ đô Hà Nội_xã đông la
1,NaN,NaN,Phường Dương Nội,True,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Phường Dương Nội,NaN,NaN,phuong duong noi,phường dương nội,NaN,NaN,NaN,Thủ đô Hà Nội_phường dương nội
2,NaN,NaN,Xã An Khánh,True,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Xã An Khánh,NaN,NaN,xa an khanh,xã an khánh,NaN,NaN,NaN,Thủ đô Hà Nội_xã an khánh
3,NaN,NaN,Xã La Phù,True,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Xã La Phù,NaN,NaN,xa la phu,xã la phù,NaN,NaN,NaN,Thủ đô Hà Nội_xã la phù
4,NaN,NaN,Xã Song Phương,True,An Khánh,Thủ đô Hà Nội,2,xã,105.708,20.98760,28.69,Xã Song Phương,NaN,NaN,xa song phuong,xã song phương,NaN,NaN,NaN,Thủ đô Hà Nội_xã song phương
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10561,NaN,NaN,Xã Phước Long,True,Vĩnh Phước,Tỉnh Cà Mau,3353,xã,105.410,9.39891,75.50,Xã Phước Long,NaN,NaN,xa phuoc long,xã phước long,NaN,NaN,NaN,Tỉnh Cà Mau_xã phước long
10562,NaN,NaN,Xã Vĩnh Phú Tây,True,Vĩnh Phước,Tỉnh Cà Mau,3353,xã,105.410,9.39891,75.50,Xã Vĩnh Phú Tây,NaN,NaN,xa vinh phu tay,xã vĩnh phú tây,NaN,NaN,NaN,Tỉnh Cà Mau_xã vĩnh phú tây
10563,NaN,NaN,Xã Hưng Phú,True,Vĩnh Thanh,Tỉnh Cà Mau,3354,xã,105.519,9.35997,37.37,Xã Hưng Phú,NaN,NaN,xa hung phu,xã hưng phú,NaN,NaN,NaN,Tỉnh Cà Mau_xã hưng phú
10564,NaN,NaN,Xã Vĩnh Thanh,True,Vĩnh Thanh,Tỉnh Cà Mau,3354,xã,105.519,9.35997,37.37,Xã Vĩnh Thanh,NaN,NaN,xa vinh thanh,xã vĩnh thanh,NaN,NaN,NaN,Tỉnh Cà Mau_xã vĩnh thanh


In [44]:
# Case 4b: Là phần còn lại của Case 2. Phải replace value và tạo lại key
sapnhap4b = df_sapnhap_ward[(~df_sapnhap_ward[key2].isna()) & (~df_sapnhap_ward[key2].isin(df_danhmuc[key2]))]
sapnhap4b['district'] = np.nan
sapnhap4b['districtUd'] = np.nan
sapnhap4b[key4] = np.where((~sapnhap4b['ward'].isna()) & (sapnhap4b['district'].isna()), sapnhap4b['newProvince'] + '_' + sapnhap4b['wardLower'], np.nan)
sapnhap4b

,province,district,ward,isMergedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
181,NaN,NaN,Phường Thạch Bàn,True,Gia Lâm,Thủ đô Hà Nội,29,xã,105.949,21.0020,25.72,Phường Thạch Bàn (huyện Gia Lâm),NaN,huyện gia lâm,phuong thach ban,phường thạch bàn,NaN,Thủ đô Hà Nội_huyện gia lâm_phường thạch bàn,NaN,Thủ đô Hà Nội_phường thạch bàn
242,NaN,NaN,Phường Tây Tựu,True,Hoài Đức,Thủ đô Hà Nội,38,xã,105.705,21.0620,16.73,Phường Tây Tựu (huyện Hoài Đức),NaN,huyện hoài đức,phuong tay tuu,phường tây tựu,NaN,Thủ đô Hà Nội_huyện hoài đức_phường tây tựu,NaN,Thủ đô Hà Nội_phường tây tựu
243,NaN,NaN,Xã Tân Lập,True,Hoài Đức,Thủ đô Hà Nội,38,xã,105.705,21.0620,16.73,Xã Tân Lập (huyện Hoài Đức),NaN,huyện hoài đức,xa tan lap,xã tân lập,NaN,Thủ đô Hà Nội_huyện hoài đức_xã tân lập,NaN,Thủ đô Hà Nội_xã tân lập
259,NaN,NaN,Phường Hoàng Liệt,True,Hoàng Liệt,Thủ đô Hà Nội,40,phường,105.832,20.9615,4.04,Phường Hoàng Liệt (huyện Thanh Trì),NaN,huyện thanh trì,phuong hoang liet,phường hoàng liệt,NaN,Thủ đô Hà Nội_huyện thanh trì_phường hoàng liệt,NaN,Thủ đô Hà Nội_phường hoàng liệt
357,NaN,NaN,Xã Văn Khê,True,Mê Linh,Thủ đô Hà Nội,56,xã,105.733,21.1510,34.97,Xã Văn Khê (huyện Đan Phượng),NaN,huyện đan phượng,xa van khe,xã văn khê,NaN,Thủ đô Hà Nội_huyện đan phượng_xã văn khê,NaN,Thủ đô Hà Nội_xã văn khê
358,NaN,NaN,Xã Mê Linh,True,Mê Linh,Thủ đô Hà Nội,56,xã,105.733,21.1510,34.97,Xã Mê Linh (huyện Đan Phượng),NaN,huyện đan phượng,xa me linh,xã mê linh,NaN,Thủ đô Hà Nội_huyện đan phượng_xã mê linh,NaN,Thủ đô Hà Nội_xã mê linh
359,NaN,NaN,Xã Đại Thịnh,True,Mê Linh,Thủ đô Hà Nội,56,xã,105.733,21.1510,34.97,Xã Đại Thịnh (huyện Đan Phượng),NaN,huyện đan phượng,xa dai thinh,xã đại thịnh,NaN,Thủ đô Hà Nội_huyện đan phượng_xã đại thịnh,NaN,Thủ đô Hà Nội_xã đại thịnh
584,NaN,NaN,Phường Tây Tựu,True,Tây Tựu,Thủ đô Hà Nội,92,phường,105.739,21.0642,7.54,Phường Tây Tựu (huyện Hoài Đức),NaN,huyện hoài đức,phuong tay tuu,phường tây tựu,NaN,Thủ đô Hà Nội_huyện hoài đức_phường tây tựu,NaN,Thủ đô Hà Nội_phường tây tựu
620,NaN,NaN,Xã Đại Mạch,True,Thiên Lộc,Thủ đô Hà Nội,98,xã,105.771,21.1204,27.96,Xã Đại Mạch (huyện Mê Linh),NaN,huyện mê linh,xa dai mach,xã đại mạch,NaN,Thủ đô Hà Nội_huyện mê linh_xã đại mạch,NaN,Thủ đô Hà Nội_xã đại mạch
621,NaN,NaN,Xã Kim Nỗ,True,Thiên Lộc,Thủ đô Hà Nội,98,xã,105.771,21.1204,27.96,Xã Kim Nỗ (huyện Mê Linh),NaN,huyện mê linh,xa kim no,xã kim nỗ,NaN,Thủ đô Hà Nội_huyện mê linh_xã kim nỗ,NaN,Thủ đô Hà Nội_xã kim nỗ


In [45]:
total_sapnhap = sapnhap1.shape[0] + sapnhap2.shape[0] + sapnhap3.shape[0] + sapnhap4a.shape[0] + sapnhap4b.shape[0]
if total_sapnhap != df_sapnhap_ward.shape[0]:
    raise Exception('Chia tệp chưa MECE')

## Tiến hành merge

In [46]:
df_convert1 = pd.merge(df_danhmuc, sapnhap1, on=['newProvince', 'province', 'districtLower', 'wardLower'], how='inner', suffixes=('', '_DUPLICATED'))
df_convert2 = pd.merge(df_danhmuc, sapnhap2, on=['newProvince', 'districtLower', 'wardLower'], how='inner', suffixes=('', '_DUPLICATED'))
df_convert3 = pd.merge(df_danhmuc, sapnhap3, on=['newProvince', 'districtLower'], how='inner', suffixes=('', '_DUPLICATED'))
df_convert4a = pd.merge(df_danhmuc, sapnhap4a, on=['newProvince', 'wardLower'], how='inner', suffixes=('', '_DUPLICATED'))
df_convert4b = pd.merge(df_danhmuc, sapnhap4b, on=['newProvince', 'wardLower'], how='inner', suffixes=('', '_DUPLICATED'))

In [47]:
print(df_convert1.shape[0], sapnhap1.shape[0])
print(df_convert2.shape[0], sapnhap2.shape[0])
print(df_convert3.shape[0], sapnhap3.shape[0])
print(df_convert4a.shape[0], sapnhap4a.shape[0])
print(df_convert4b.shape[0], sapnhap4b.shape[0])


2 2
903 903
9 6
9641 9604
53 52


In [48]:
df_convert = pd.concat([df_convert1, df_convert2, df_convert3, df_convert4a, df_convert4b], ignore_index=True).drop_duplicates()
df_convert.drop(columns=[col for col in df_convert.columns if '_DUPLICATED' in col or 'key' in col], inplace=True)
df_convert.drop_duplicates(inplace=True)

In [49]:
df_convert[df_convert['wardUd'].fillna('').str.contains('phuong 15') & df_convert['districtUd'].fillna('').str.contains('tan binh')]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,...,wardUd,wardLower,isMergedWard,newWard,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap
601,766,79,Quận,Quận Tân Bình,Thành phố Hồ Chí Minh,27007.0,Phường,Phường 15,29,Thành phố Hồ Chí Minh,...,phuong 15,phường 15,True,Tân Bình,2807,phường,106.642,10.8006,2.12,Phường 15 (quận Tân Bình)
602,766,79,Quận,Quận Tân Bình,Thành phố Hồ Chí Minh,27007.0,Phường,Phường 15,29,Thành phố Hồ Chí Minh,...,phuong 15,phường 15,True,Tân Sơn,2819,phường,106.650,10.8224,10.12,Phường 15 (quận Tân Bình)


In [50]:
df_convert[df_convert['newWardCode']==3257]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,...,wardUd,wardLower,isMergedWard,newWard,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap
10169,925,92,Huyện,Huyện Cờ Đỏ,Thành phố Cần Thơ,31249.0,Xã,Xã Thạnh Phú,33,Thành phố Cần Thơ,...,xa thanh phu,xã thạnh phú,False,Thạnh Phú,3257,xã,105.405,10.1295,99.07,Không sáp nhập
10340,947,94,Huyện,Huyện Mỹ Xuyên,Tỉnh Sóc Trăng,31708.0,Xã,Xã Thạnh Phú,33,Thành phố Cần Thơ,...,xa thanh phu,xã thạnh phú,False,Thạnh Phú,3257,xã,105.405,10.1295,99.07,Không sáp nhập


In [51]:
df_convert_full = pd.merge(df_danhmuc, df_convert, on=['newProvince', 'province', 'districtLower', 'wardLower'], how='left', suffixes=('', '_DUPLICATED')).drop_duplicates()
df_convert_full .drop(columns=[col for col in df_convert_full.columns if '_DUPLICATED' in col or 'key' in col], inplace=True)
df_convert_full.drop_duplicates(inplace=True)

In [52]:
df_convert_full[df_convert_full['newWardCode']==3257]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,...,wardUd,wardLower,isMergedWard,newWard,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap
10197,925,92,Huyện,Huyện Cờ Đỏ,Thành phố Cần Thơ,31249.0,Xã,Xã Thạnh Phú,33,Thành phố Cần Thơ,...,xa thanh phu,xã thạnh phú,False,Thạnh Phú,3257.0,xã,105.405,10.1295,99.07,Không sáp nhập
10374,947,94,Huyện,Huyện Mỹ Xuyên,Tỉnh Sóc Trăng,31708.0,Xã,Xã Thạnh Phú,33,Thành phố Cần Thơ,...,xa thanh phu,xã thạnh phú,False,Thạnh Phú,3257.0,xã,105.405,10.1295,99.07,Không sáp nhập


In [53]:
print('df_convert_full.shape:', df_convert_full.shape)

df_convert_full.shape: (10610, 25)


In [54]:
df_convert_full[df_convert_full['newWard'].isna()]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,...,wardUd,wardLower,isMergedWard,newWard,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap
959,34,2,Huyện,Huyện Bắc Quang,Tỉnh Hà Giang,1174.0,Xã,Xã Thượng Bình,8,Tỉnh Tuyên Quang,...,xa thuong binh,xã thượng bình,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1358,75,8,Huyện,Huyện Yên Sơn,Tỉnh Tuyên Quang,2488.0,Xã,Xã Trung Môn,8,Tỉnh Tuyên Quang,...,xa trung mon,xã trung môn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
new_province = 'Thành phố Cần Thơ'
ward_ud = 'thanh phu'
df_sapnhap_ward[(df_sapnhap_ward['newProvince'] == new_province) & (df_sapnhap_ward['wardUd'].fillna('').str.contains(ward_ud))]

,province,district,ward,isMergedWard,newWard,newProvince,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
10236,NaN,NaN,Xã Thạnh Phú,True,Nhu Gia,Thành phố Cần Thơ,3234,xã,105.855,9.47039,75.59,Xã Thạnh Phú,NaN,NaN,xa thanh phu,xã thạnh phú,NaN,NaN,NaN,Thành phố Cần Thơ_xã thạnh phú
10296,NaN,NaN,Xã Thạnh Phú,False,Thạnh Phú,Thành phố Cần Thơ,3257,xã,105.405,10.12950,99.07,Không sáp nhập,NaN,NaN,xa thanh phu,xã thạnh phú,NaN,NaN,NaN,Thành phố Cần Thơ_xã thạnh phú


In [56]:
df_danhmuc[(df_danhmuc['newProvince'] == new_province) & (df_danhmuc['wardUd'].fillna('').str.contains(ward_ud))]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,...,newProvinceLon,isMergedProvince,districtUd,districtLower,wardUd,wardLower,keyNewProvinceProvinceDistrictWard,keyNewProvinceDistrictWard,keyNewProvinceDistrict,keyNewProvinceWard
9665,925,92,Huyện,Huyện Cờ Đỏ,Thành phố Cần Thơ,31249.0,Xã,Xã Thạnh Phú,33,Thành phố Cần Thơ,...,105.757,True,huyen co do,huyện cờ đỏ,xa thanh phu,xã thạnh phú,Thành phố Cần Thơ_Thành phố Cần Thơ_huyện cờ đ...,Thành phố Cần Thơ_huyện cờ đỏ_xã thạnh phú,Thành phố Cần Thơ_huyện cờ đỏ,Thành phố Cần Thơ_xã thạnh phú
9829,947,94,Huyện,Huyện Mỹ Xuyên,Tỉnh Sóc Trăng,31708.0,Xã,Xã Thạnh Phú,33,Thành phố Cần Thơ,...,105.757,True,huyen my xuyen,huyện mỹ xuyên,xa thanh phu,xã thạnh phú,Thành phố Cần Thơ_Tỉnh Sóc Trăng_huyện mỹ xuyê...,Thành phố Cần Thơ_huyện mỹ xuyên_xã thạnh phú,Thành phố Cần Thơ_huyện mỹ xuyên,Thành phố Cần Thơ_xã thạnh phú


In [57]:
df_convert_full[(df_convert_full['newProvince'] == new_province) & (df_convert_full['wardUd'].fillna('').str.contains(ward_ud))]

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,...,wardUd,wardLower,isMergedWard,newWard,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap
10196,925,92,Huyện,Huyện Cờ Đỏ,Thành phố Cần Thơ,31249.0,Xã,Xã Thạnh Phú,33,Thành phố Cần Thơ,...,xa thanh phu,xã thạnh phú,True,Nhu Gia,3234.0,xã,105.855,9.47039,75.59,Xã Thạnh Phú
10197,925,92,Huyện,Huyện Cờ Đỏ,Thành phố Cần Thơ,31249.0,Xã,Xã Thạnh Phú,33,Thành phố Cần Thơ,...,xa thanh phu,xã thạnh phú,False,Thạnh Phú,3257.0,xã,105.405,10.12950,99.07,Không sáp nhập
10373,947,94,Huyện,Huyện Mỹ Xuyên,Tỉnh Sóc Trăng,31708.0,Xã,Xã Thạnh Phú,33,Thành phố Cần Thơ,...,xa thanh phu,xã thạnh phú,True,Nhu Gia,3234.0,xã,105.855,9.47039,75.59,Xã Thạnh Phú
10374,947,94,Huyện,Huyện Mỹ Xuyên,Tỉnh Sóc Trăng,31708.0,Xã,Xã Thạnh Phú,33,Thành phố Cần Thơ,...,xa thanh phu,xã thạnh phú,False,Thạnh Phú,3257.0,xã,105.405,10.12950,99.07,Không sáp nhập


In [58]:
wrong_index_1 = df_convert_full[(df_convert_full['newProvince'] == 'Thành phố Cần Thơ') & (df_convert_full['wardUd'].fillna('').str.contains('thanh phu'))  & (df_convert_full['province']=='Tỉnh Sóc Trăng') & (df_convert_full['isMergedWard']==False)].index[0]

wrong_index_2 = df_convert_full.loc[(df_convert_full['newProvince'] == 'Thành phố Cần Thơ') & (df_convert_full['wardUd'].fillna('').str.contains('thanh phu'))  & (df_convert_full['province']=='Thành phố Cần Thơ') & (df_convert_full['isMergedWard']==True)].index[0]

In [59]:
df_convert_full  = df_convert_full[~df_convert_full.index.isin([wrong_index_1, wrong_index_2])]

In [60]:
# Cần xóa đi các ward đã là isMergedWard = True rồi mà vẫn có một dòng isMergedWard = False
# Vì những case mà cùng newProvince nhưng ward bị trùng tên với một ward ở district khác (Tỉnh Quảng Ngãi, Xã Đăk Long)
# isMergedWard là boolean. Chúng ta sẽ tính COUNT và SUM
# SUM = 0 hoặc 1 và COUNT = 1: Là đúng (False vẫn COUNT là 1, nhưng SUM là 0)
# SUM = 1 mà COUNT = 2: Sai -> Lọc lấy những ward này, xóa dòng mà nó có isMergedWard = False
agg_isMergedWard = df_convert_full[['province', 'district', 'ward', 'isMergedWard']].drop_duplicates()
agg_isMergedWard['isMergedWard'] = agg_isMergedWard['isMergedWard'].map({True: 1, False: 0})
agg_isMergedWard = agg_isMergedWard.groupby(['province', 'district', 'ward']).agg(sumIsMergedWard=('isMergedWard', 'sum'), countIsMergedWard=('isMergedWard', 'count')).reset_index()

wrong_ward = agg_isMergedWard[(agg_isMergedWard['sumIsMergedWard']==1) & (agg_isMergedWard['countIsMergedWard']==2)]
wrong_ward['isMergedWard'] = False
wrong_ward['wrongWard'] = True
wrong_ward.drop(columns=['sumIsMergedWard', 'countIsMergedWard'], inplace=True)

df_convert_full = pd.merge(df_convert_full, wrong_ward, on=['province', 'district', 'ward', 'isMergedWard'], how='left')
df_convert_full = df_convert_full[df_convert_full['wrongWard']!=True]

In [61]:
# Điền vào ở bước tiếp theo
not_exist_in_sapnhap = [
    'Xã Thượng Bình', # newProvince = "Tỉnh Tuyên Quang" -> newWard = "Xã Đồng Tâm" https://address-converter.io.vn/
    'Xã Trung Môn', # newProvince = "Tỉnh Tuyên Quang" -> newWard = "Phường Minh Xuân" https://address-converter.io.vn/
]

# Quay lại bước kiểm tra chính tả 2 bên để bổ sung. Xuân Hòa và Phú Bình trong Thành phố Long Khánh là Phường mới đúng, sửa df_sapnhap_ward
exist_in_sapnhap = [
    'Phường Xuân Hòa', # ward = 'Xã Xuân Hòa' newWard = 'Long Khánh', newProvince = 'Tỉnh Đồng Nai'
    'Phường Phú Bình', # ward = 'Xã Phú Bình' newWard = 'Long Khánh', newProvince = 'Tỉnh Đồng Nai'
]

In [62]:
fill_map = [
    {
        'province': 'Tỉnh Hà Giang',
        'district': 'Huyện Bắc Quang',
        'ward': 'Xã Thượng Bình',
        'newProvince': 'Tỉnh Tuyên Quang',
        'newWard': 'Đồng Tâm'
    },
    {
        'province': 'Tỉnh Tuyên Quang',
        'district': 'Huyện Yên Sơn',
        'ward': 'Xã Trung Môn',
        'newProvince': 'Tỉnh Tuyên Quang',
        'newWard': 'Minh Xuân'
    },
]
fill_cols = ['isMergedWard', 'newWard', 'newWardCode', 'newWardType', 'newWardLon', 'newWardLat', 'newWardAreaKm2']
for record in fill_map:
    miss_condition = (df_convert_full['province'] == record['province']) & (df_convert_full['district'] == record['district']) & (df_convert_full['ward'] == record['ward'])
    fill_condition = (df_convert_full['newProvince'] == record['newProvince']) & (df_convert_full['newWard'] == record['newWard'])

    fill_values = df_convert_full.loc[fill_condition, fill_cols].iloc[0]
    df_convert_full.loc[miss_condition, fill_cols] = fill_values.values

In [63]:
# df_convert_full.to_csv('data/danhmuc_and_sapnhap.csv', index=False)

In [64]:
# Tính isDividedWard
count_unit = df_convert_full.groupby(['province', 'district', 'ward']).size().reset_index(name='count_unit')
count_unit['isDividedWard'] = np.where(count_unit['count_unit'] > 1, True, False)
df_convert_full = pd.merge(df_convert_full, count_unit[['province', 'district', 'ward', 'isDividedWard']], on=['province', 'district', 'ward'], how='left')
df_convert_full['isDividedWard'].fillna(False, inplace=True)

In [65]:
df_convert_full

,districtCode,provinceCode,districtType,district,province,wardCode,wardType,ward,newProvinceCode,newProvince,...,isMergedWard,newWard,newWardCode,newWardType,newWardLon,newWardLat,newWardAreaKm2,truocsapnhap,wrongWard,isDividedWard
0,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,1.0,Phường,Phường Phúc Xá,1,Thủ đô Hà Nội,...,True,Hồng Hà,42.0,phường,105.845,21.05670,15.09,Phường Phúc Xá,NaN,False
1,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,4.0,Phường,Phường Trúc Bạch,1,Thủ đô Hà Nội,...,True,Ba Đình,3.0,phường,105.838,21.03860,2.97,Phường Trúc Bạch,NaN,False
2,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,6.0,Phường,Phường Vĩnh Phúc,1,Thủ đô Hà Nội,...,True,Ngọc Hà,61.0,phường,105.816,21.03810,2.68,Phường Vĩnh Phúc,NaN,False
3,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,7.0,Phường,Phường Cống Vị,1,Thủ đô Hà Nội,...,True,Giảng Võ,30.0,phường,105.814,21.02750,2.60,Phường Cống Vị,NaN,True
4,1,1,Quận,Quận Ba Đình,Thành phố Hà Nội,7.0,Phường,Phường Cống Vị,1,Thủ đô Hà Nội,...,True,Ngọc Hà,61.0,phường,105.816,21.03810,2.68,Phường Cống Vị,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10597,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32242.0,Xã,Xã Viên An,34,Tỉnh Cà Mau,...,True,Đất Mũi,3302.0,xã,104.820,8.63765,271.20,Xã Viên An,NaN,True
10598,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32244.0,Thị trấn,Thị trấn Rạch Gốc,34,Tỉnh Cà Mau,...,True,Phan Ngọc Hiển,3327.0,xã,104.943,8.64616,237.70,Thị trấn Rạch Gốc,NaN,False
10599,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32245.0,Xã,Xã Tân Ân,34,Tỉnh Cà Mau,...,True,Đất Mũi,3302.0,xã,104.820,8.63765,271.20,Xã Tân Ân,NaN,True
10600,973,96,Huyện,Huyện Ngọc Hiển,Tỉnh Cà Mau,32245.0,Xã,Xã Tân Ân,34,Tỉnh Cà Mau,...,True,Phan Ngọc Hiển,3327.0,xã,104.943,8.64616,237.70,Xã Tân Ân,NaN,True


In [66]:
save_cols = [
    'provinceCode',
    'isMergedProvince',
    'districtCode',
    'districtType',
    'wardCode',
    'wardType',
    'isMergedWard',
    'isDividedWard',

    'province',
    'district',
    'ward',

    'newProvince',
    'newWard',

    'newProvinceCode',
    'newProvinceLat',
    'newProvinceLon',
    'newWardCode',
    'newWardType',
    'newWardLat',
    'newWardLon',
    'newWardAreaKm2',
]

In [67]:
df_convert_full = df_convert_full[save_cols].drop_duplicates()

In [68]:
df_convert_full

,provinceCode,isMergedProvince,districtCode,districtType,wardCode,wardType,isMergedWard,isDividedWard,province,district,...,newProvince,newWard,newProvinceCode,newProvinceLat,newProvinceLon,newWardCode,newWardType,newWardLat,newWardLon,newWardAreaKm2
0,1,False,1,Quận,1.0,Phường,True,False,Thành phố Hà Nội,Quận Ba Đình,...,Thủ đô Hà Nội,Hồng Hà,1,21.0001,105.698,42.0,phường,21.05670,105.845,15.09
1,1,False,1,Quận,4.0,Phường,True,False,Thành phố Hà Nội,Quận Ba Đình,...,Thủ đô Hà Nội,Ba Đình,1,21.0001,105.698,3.0,phường,21.03860,105.838,2.97
2,1,False,1,Quận,6.0,Phường,True,False,Thành phố Hà Nội,Quận Ba Đình,...,Thủ đô Hà Nội,Ngọc Hà,1,21.0001,105.698,61.0,phường,21.03810,105.816,2.68
3,1,False,1,Quận,7.0,Phường,True,True,Thành phố Hà Nội,Quận Ba Đình,...,Thủ đô Hà Nội,Giảng Võ,1,21.0001,105.698,30.0,phường,21.02750,105.814,2.60
4,1,False,1,Quận,7.0,Phường,True,True,Thành phố Hà Nội,Quận Ba Đình,...,Thủ đô Hà Nội,Ngọc Hà,1,21.0001,105.698,61.0,phường,21.03810,105.816,2.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10597,96,True,973,Huyện,32242.0,Xã,True,True,Tỉnh Cà Mau,Huyện Ngọc Hiển,...,Tỉnh Cà Mau,Đất Mũi,34,9.1362,105.182,3302.0,xã,8.63765,104.820,271.20
10598,96,True,973,Huyện,32244.0,Thị trấn,True,False,Tỉnh Cà Mau,Huyện Ngọc Hiển,...,Tỉnh Cà Mau,Phan Ngọc Hiển,34,9.1362,105.182,3327.0,xã,8.64616,104.943,237.70
10599,96,True,973,Huyện,32245.0,Xã,True,True,Tỉnh Cà Mau,Huyện Ngọc Hiển,...,Tỉnh Cà Mau,Đất Mũi,34,9.1362,105.182,3302.0,xã,8.63765,104.820,271.20
10600,96,True,973,Huyện,32245.0,Xã,True,True,Tỉnh Cà Mau,Huyện Ngọc Hiển,...,Tỉnh Cà Mau,Phan Ngọc Hiển,34,9.1362,105.182,3327.0,xã,8.64616,104.943,237.70


In [69]:
if df_convert_full['newWard'].isna().sum():
    raise Exception('Missing new wards')

In [70]:
if df_convert_full[['province', 'district', 'ward']].drop_duplicates().shape[0] != df_danhmuc.shape[0]:
    raise Exception('Convert missing ward in danhmuc')

In [71]:
if df_convert_full[['newProvince', 'newWard', 'newWardCode']].drop_duplicates().shape[0] != 3321:
    raise Exception('Convert missing ward in sapnhap')

In [72]:
df_convert_full.to_csv(BASE_DIR / 'data/interim/convert_legacy_2025_simple.csv', index=False)